In [9]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [10]:
df=pd.read_csv('netflix_titles.csv')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [11]:
# !pip install transformers
# !pip install  sentence_transformers

In [12]:
import torch
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('paraphrase-distilroberta-base-v1')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
descriptions = df['description'].tolist()

des_embeddings = []
for i,des in enumerate(descriptions):
    des_embeddings.append(model.encode(des))





In [14]:
import torch
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Example: Initialize model
model = SentenceTransformer('paraphrase-distilroberta-base-v1')


# Example: Initialize des_embeddings (mocked)
des_embeddings = torch.randn((len(df), 768))  # Replace with actual embeddings

def recommend(query_title, df):
    # Get show description based on title
    query_title = query_title.strip()  # Remove leading/trailing spaces
    query_show_des = df[df['title'].str.strip() == query_title]['description'].to_list()[0]

    # Encode the query description and calculate cosine similarities
    query_embedd = model.encode(query_show_des)
    cosine_scores = util.pytorch_cos_sim(query_embedd, des_embeddings)

    # Find top 5 most similar shows (titles)
    top5_matches = torch.argsort(cosine_scores, dim=-1, descending=True).tolist()[0][1:6]
    similar_titles = df.iloc[top5_matches]['title'].to_list()

    # Create and return DataFrame
    result_df = pd.DataFrame({'title': similar_titles})

    # Save embeddings (des_embeddings) to a file
    torch.save(des_embeddings, 'des_embeddings.pt')

    return result_df



In [15]:
# Example usage (assuming `netflix_data` and `model` are defined elsewhere)
query_title = "Stranger Things"
recommended_df = recommend(query_title, df)
print(recommended_df)

                                               title
0                                   Hole in the Wall
1                                          Moonlight
2  Teenage Mutant Ninja Turtles II: The Secret of...
3           Chico Bon Bon and the Very Berry Holiday
4                                     A Whisker Away


In [16]:
# Print df['description'] against each title of recommended_df
for index, row in recommended_df.iterrows():
    title = row['title']
    description = df[df['title'] == title]['description'].values[0]
    print(f"Title: {title}")
    print(f"Description: {description}\n")

Title: Hole in the Wall
Description: Diagnosed with colon cancer, a free-spirited man embarks on an illuminating road trip with his son through South Africa.

Title: Moonlight
Description: In a crime-infested Miami neighborhood, a gay African-American boy with a drug-addicted mother grows up with the help of a mentoring drug dealer.

Title: Teenage Mutant Ninja Turtles II: The Secret of the Ooze
Description: The evil Shredder decides that ooze is what gives the ninja turtles their power, so he obtains some of the toxic junk and creates dangerous mutants.

Title: Chico Bon Bon and the Very Berry Holiday
Description: The Fix-It Force makes a plan to hit every home as fast as they can, delivering Blunderberry Cakes before the town awakes to avoid a holiday disaster.

Title: A Whisker Away
Description: A peculiar girl transforms into a cat to catch her crush's attention. But before she realizes it, the line between human and animal starts to blur.



In [27]:
# Example usage (assuming `netflix_data` and `model` are defined elsewhere)
query_title = "Kai Po Che!"
recommended_df = recommend(query_title, df)
print(recommended_df)

                                    title
0                        Would You Rather
1                 The Man Without Gravity
2                              Malevolent
3                               The Brave
4  Trailer Park Boys: The Animated Series


In [28]:
# Print df['description'] against each title of recommended_df
for index, row in recommended_df.iterrows():
    title = row['title']
    description = df[df['title'] == title]['description'].values[0]
    print(f"Title: {title}")
    print(f"Description: {description}\n")

Title: Would You Rather
Description: In need of cash to help her sick brother, a young woman agrees to take part in a lethal winner-takes-all parlor game hosted by a sadistic millionaire.

Title: The Man Without Gravity
Description: A gravity-defying boy raised in seclusion matures into an extraordinary man – and an international celebrity – who longs for human connection.

Title: Malevolent
Description: A brother-sister team who fake paranormal encounters for cash get more than they bargained for when a job at a haunted estate turns very, very real.

Title: The Brave
Description: With entry into the European Union at stake, Albania must rely on a mash-up of elite cops to take down a slippery drug lord dominating its economy.

Title: Trailer Park Boys: The Animated Series
Description: Nova Scotia’s favorite miscreants have always been super sketchy. Now, carrying on from the Season 12 finale, the boys have become complete cartoons.

